In this notebook, we will implement a Stochastic ResNet architecture for image classification. Stochastic ResNets introduce randomness in the forward pass by propagating uncertainty alongside features, enabling better regularization and improved generalization through stochastic depth.